In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import SummaryIndex
from llama_index.core import Settings
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.query_engine import SubQuestionQueryEngine

from qazure import get_llm, get_embedder

import logging
import sys

In [2]:
logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
llm = get_llm()
embed_model = get_embedder()

In [4]:
Settings.llm = llm
Settings.embed_model = embed_model

In [5]:
# Simple query by indexing
# TODO: Try multimedia inputs like mp3 and mp4
documents = SimpleDirectoryReader(
    input_files=["feedback.docx"]
).load_data()
feedback_index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"


In [6]:
query = "What is the feedback about? Give me names."
query_engine = feedback_index.as_query_engine()
answer = query_engine.query(query)

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"


In [7]:
print(answer.get_formatted_sources())
print("Query was:", query)
print("Answer was:", answer)

> Source (Doc id: 089e36ff-0279-4a5c-9534-c8222ff45c4e): Interview Feedback Form - Qylis

Section 1: Candidate Details 

Name of Candidate   : ______ ____...
Query was: What is the feedback about? Give me names.
Answer was: The feedback is about a candidate named Sohel, who applied for a Data Science position and was interviewed by Akshay.


In [30]:
# Summarization
summary_index = SummaryIndex.from_documents(documents)

query_engine = summary_index.as_query_engine(response_mode="tree_summarize", streaming=True)
response = query_engine.query("<summarization_query>")
response.print_response_stream()
print('')

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
The interview feedback form for Sohel, who applied for a Data Science position, evaluates technical, behavioral, and cultural fit criteria. Strengths noted include openness and the ability to quickly grasp concepts, while areas for improvement include a better understanding of technical aspects in the field. No additional comments were provided, and the overall recommendation is left unspecified.


In [32]:
documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()
essay_index = VectorStoreIndex.from_documents(documents)

# Define query engines and tools
tool1 = QueryEngineTool.from_defaults(
    query_engine=feedback_index.as_query_engine(),
    description="Use this query engine for feedback form.",
    # description="Use this query engine for an English essay.", Switch
)
tool2 = QueryEngineTool.from_defaults(
    query_engine=essay_index.as_query_engine(),
    description="Use this query engine for an English essay.",
    # description="Use this query engine for feedback form.", Switch
)

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"


In [8]:
# Get as chat. Keeps track of conversation history
chat_engine = feedback_index.as_chat_engine()
chat1 = "Who is the interview candidate?"
response1 = chat_engine.chat(chat1)
print('Response1:', response1)
chat2 = "Who is the interviewer?"
response2 = chat_engine.chat(chat2)
print('Response2:', response2)

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
Response1: Could you provide more context or details about the interview candidate you're referring to? Are you asking about a specific interview or a general concept?
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
Response2: The interviewer is the person conducting the interview. They are typically responsible for asking questions, assessing the candidate's qualifications, and determining whether the candidate is a good fit 

In [13]:
chat3 = "Give me the summary of the feedback?"
response3 = chat_engine.chat(chat3)
print(response3)

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.op

In [ ]:
# For interactive chat
# chat_engine.chat_repl()